In [ ]:
# helpers
from migen.fhdl.verilog import convert
import tempfile
from IPython.display import SVG, display

def verilog2blocsvg(vlog):
    with open('m.v', 'w') as f:
        f.write(str(vlog)) 
    hideoutput = !symbolator -i m.v
    return SVG('m-top.svg')


def module2blocsvg(module, ios):
    return verilog2svg(convert(m, ios=ios))

def str2tmpfile(s, suffix):
    with tempfile.NamedTemporaryFile(mode='w', buffering=-1, suffix=suffix, delete=False) as f:
        name = f.name
        f.write(str(s))
    return name

# What are FPGA?

**F**ield **P**rogrammable (logic) **G**ate **A**rray

Basically it's a whiteboard on which you can draw (nearly) any kind of logic.
Contrary to SoC or CPUs you can buy on the market, where a single CPU runs (almost) sequentially each instruction,  everything can run in parallel in an FPGA.  
An FPGA has a grid-like combination of multiple **Logic Cell** (sometimes called Logic Blocks), and more specialized blocks (memory, DSP, interface, clocks)

## Lattice's ECP5

 * Low to mid-range FPGA
   * simple, thus easy to master
   * affordable (a 6€ FPGA can run Linux on a RISC-V CPU)
   * but relatively slow (50-100MHz) and small (25k to 85k logic cells)
 * The biggest available FPGA with good sopport of 100% FOSS toolchain

Let's dig down into the datasheet : http://www.latticesemi.com/-/media/LatticeSemi/Documents/DataSheets/ECP5/FPGA-DS-02012-1-9-ECP5-ECP5G-Family-Data-Sheet.ashx?document_id=50461

# Hardware Description Language
FPGA don't "run code", you describe the logic that runs into it (code run sequentially, HDL in parallel).  
The toolchain then creates (one of) the configuration that behaves as described.  

In the FPGA world, 99% of the logic is either **combinatory** or **synchronous**. They are essentially the same, but the synchronous logic have an extra small memory (flip-flop) that 'lock' the output value and keeps it stable until the next clock cycle.  

These flip-flops help closing timing constraints by splitting a long logic path

# Workflow
Nowadays, most toolchains uses roughly the same steps (and terminology) for building a complete FPGA configuration from an HDL code.
   
Let's have a look how [symbiflow](https://symbiflow.readthedocs.io/en/latest/toolchain-desc/design-flow.html) does it.

### 1. Synthesis  
   1.1. RTL generation: convert HDL into a machine representation made with abstract blocks (logic, registers) and connection between them. That's *Register-Transfer level*. Some reduction logic optimization are usually aplied at this stage like [truth table reduction](https://learnabout-electronics.org/Digital/dig24.php)   
   1.2. Technology mapping : FPGAs comprise different logic elements, or specialized elements like DSP and RAM. At this stage, the toolchain translates the RTL into configured elements, which assembled together behave like the RTL describes it.  
   1.3. Optimization : Unused signals and logic are removed. Some toolchains can also merge identical signals from different modules.

### 2. Place & Route    
   2.1. Packing : "pre-place" small parts of the design, roughly on the logic-cell level  
   2.2. Placing : attributes a physical location to each of the packed pieces. Since routing of signals between cells is limited, this has a big impact on routing quality  
   2.3. Routing : the tool has to find a solution that makes all connections successfully, but at the same time it must respect timing criterias (shorter connection is faster)  
   2.4. Analysis: checks that the design is valid, that the timing specs are respected 

# Before we continue, let's install the toolchain for ECP5 / ICE40

For Manjaro:
```
yay boost cmake openocd qt5 libeigen3
git clone https://github.com/SymbiFlow/prjtrellis.git # ECP5 'layout' database + bitstream generation tool
git clone https://github.com/YosysHQ/nextpnr.git # placer / router 
git clone https://github.com/YosysHQ/yosys.git # synthetizer
cd prjtrellis/libtrellis
cmake -DCMAKE_INSTALL_PREFIX=/usr . && make
sudo make install
cd ../../yosys
make
sudo make install
cd ../nextpnr
cmake -DARCH=ecp5 . && make
sudo make install
pip install git+https://github.com/chmousset/litex.git@faae7fdb5ac23e9dfad31279f90338427bdcb805
pip install migen
```

# Before we continue, let's install the toolchain for ECP5 / ICE40

For 'native' Ubuntu users, `https://github.com/ulx3s/ulx3s-toolchain` can be used:
```
cd ~
wget https://raw.githubusercontent.com/ulx3s/ulx3s-toolchain/master/install.sh
chmod +x install.sh
./install.sh barebones aptget
```
or, you can use the VM: https://www.dropbox.com/s/s5f4h5gbi9bbg78/fpga_workshop.ova?dl=0  

Either way, you will need a fix in LiteX to support the 12k FPGA from the ULX3s:
`pip install --user --upgrade git+https://github.com/chmousset/litex@faae7fdb5ac23e9dfad31279f90338427bdcb805`

In [ ]:
# We will create a trivial module to illustrate workflow steps
from migen import *
from migen.fhdl.verilog import convert

class counter(Module):
    def __init__(self, width, maxcnt, platform):
        self.platform = platform
        y = Signal(width)
        self.sync += [
            If((y == maxcnt-1),
              y.eq(0),
            ).Else(
                y.eq(y+1),
            ),
        ]
        self.ios = {y}

m = counter(5, 14, None)
vlog = convert(m, ios=m.ios)
f_counter = str2tmpfile(vlog, '.v')
print(vlog)
display(verilog2blocsvg(vlog))

In [ ]:
# Step 1.1 : RTL generation
# We follow techniques from http://www.clifford.at/yosys/files/yosys_appnote_011_design_investigation.pdf
ys = """
    read_verilog {}
    show
    """.format(f_counter)
a = !yosys {str2tmpfile(ys, '.ys')}
print('\n'.join(a))

In [ ]:
# Step 1.2 : technology mapping (generic)
# This replaces the processes in the design with multiplexers, flip-flops and latches.
ys = """
    read_verilog {}
    proc
    show
    """.format(f_counter)
a = !yosys {str2tmpfile(ys, '.ys')}
print('\n'.join(a))

In [ ]:
# Step 1.3 : Optimization
ys = """
    read_verilog {}
    proc
    opt
    show
    """.format(f_counter)
a = !yosys {str2tmpfile(ys, '.ys')}
print('\n'.join(a))

In [ ]:
# Step 2 : pack, place, route
!nextpnr-ecp5 --gui --25k --package CABGA381

# Migen
FHDL DSL in Python "A Python toolbox for building complex digital hardware"
* More restrictive than pure Verilog / VHDL (targeted mostly at FPGA)
* HDL itself isn't really better than any other options (except that it lets you do less mistakes)
* The true power comes from flexibility and modularity
* it's an HDL *generator*!

`python -m pip install --user git+https://github.com/m-labs/migen.git`

## Modules

Combinatory and synchronous logic takes place inside Modules.
The 'main' module (called the 'top' module) can have `submodules`, which are also `Module` instances. There can be only a single 'top' module, so your designs is essentially a `Module` tree. Anything outside the tree's `comb` and `sync` won't generate any logic.

Module hiearchy helps segmenting and organizing an FPGA design per it's subsystems.

NB:  
* All of every modules's `comb` and `sync` statements are running in parallel.
* Inputs and outputs are not always very clear in Migen. You can put them in constructor argument but you don't have to 
* Modules are HDL **generators**

In [ ]:
from migen import *

class mymodule(Module):
    def __init__(self, a, b, difference):
        y = Signal()
        y_ff = Signal()
        self.comb += [
            y.eq(a + b),
            difference.eq(a-b)
        ]
        self.sync += [
            y_ff.eq(a+b)
        ]
        useless = Signal() # not used anywhere, won't generate logic
        
        self.y = y # will be acessible by parent Module
        # y_ff is not accessible by parent Module, thus, can't be connected to IOs or other modules.
        # it will be removed with its associated logic during the optimization phase

# Signals
A `Signal` object carries 1 or multiple 'bits' of information.
 * Fixed size, determined before synthetisation (you know what size it should be beforehand)
 * Fixed size, determined at synthetisation (the toolchain guesses from where the signal comes from)
 * Can be signed (optional)
 * represents either an input, output, the result of a combinatory circuit, or the value of a register (automatically determined by Migen)  
 * `Constant` is the constant flavor

In [ ]:
from migen import Signal
a = Signal() # 1 bit signal
b = Signal(2) # 2 bits signal
c = Signal(max=32) # 5 bits signal (NOT 6!)
print(c, c.nbits)
d = Signal(reset=3) # 2 bits signal, reset (default) value set to 3
print(d, d.nbits)
e = Signal(name="mybeautyfulsignal", max=6)
print(e, e.nbits)
e = Signal(name="mybeautyfulsignedsignal", min=-6, max=6) # sign takes 1 bit
print(e, len(e))
f = Signal.like(e)

## Slicing
Using python slice
* `[2]` : select only the 3rd lowest-significant bit
* `[0:-4]` : select 1sth lowest, to 4th highest bit
* `[1:3]` : select 2nd and 3rd lowest bits (and not 2nd, 3rd and 4th lowest like in Verilog!)
* etc

In [ ]:
s = Signal(32, reset=0xdeadbeef)
slic = s[0:4]
print(slic, len(slic))

## Operators
`*`, `-`, `and`, `^` etc. can be used directly supported. It creates an `_Operator` object which can be used in place of a `Signal`

In [ ]:
op = a*b + 3
print(op.operands, len(op))
print(op.operands[0].operands)

# Concatenation, Replication
`Cat()` stitches multiple `Signals` or `Constant` together.  
`Replicate(0, 4)` is equivalent to `Cat(0, 0, 0, 0)`

**Concatenation is done from lowest to highest bit, contrary to Verilog's `{}` operator!**

```
a = Signal() # 1 bit signal
b = Signal(2) # 2 bits signal
c = Cat(a, b)
# c[0] = a
# c[1:] = b
```

## Assignment
*It's your `=` from the programming world.* Can only be done inside combinatory or synchronous blocks (otherwise has no effect).

**NOTE: assignments are all running in parallel in `sync` blocs!**

```
sync += [
    a.eq(a+1),
    b.eq(a),
]
```
is exactly the same as 
```

sync += [
    b.eq(a),
    a.eq(a+1),
]
```

## If, Elif, Else
Functions `If()`, `Elif()` and `Else()` return statement objects.

The first parameter of If and Elif is a migen value (`Signal`, `Constant`, `_Slice` etc).
The other argument(s) are statements

In [ ]:
If(a == 1,
    b.eq(1),
).Elif(op > 5,
    b.eq(2),
).Else(
    b.eq(3),
    slic.eq(32),
)

In [ ]:
If(a,
    If(b,
        c.eq(1),
    ),
)

## Case, Array, Instance, specials ...
=> see FHDL documentation https://m-labs.hk/migen/manual/fhdl.html

# Verilog / Migen comparison
Let's create a 'BCD to ACSII' converter as an example

In [ ]:
from migen import Module, Case
class bcd2ascii_ifs(Module):
    def __init__(self, bcd):
        self.char = Signal(8)
        self.comb += [
            self.char.eq(ord('x')), # set the default value to 'undefined' ('high impedance' for external IOs)
            If(bcd == 0,
                self.char.eq(ord('0'))
            ),
            If(bcd==1,
                self.char.eq(ord('1'))
            ),
            If(bcd==2,
                self.char.eq(ord('2'))
            )
            # and so on ...
        ]

# Let's convert this to Verilog!
from migen.fhdl.verilog import convert
bcd = Signal(4)    # input signal
m = bcd2ascii_ifs(bcd) # module instance
print(convert(m, ios={m.char, bcd}))

In [ ]:
from migen import Module, Case
class bcd2ascii_case(Module):
    def __init__(self, bcd):
        self.char = Signal(8)
        self.comb += Case(bcd, {i: self.char.eq(ord(c)) for i,c in enumerate("0123456789abcdef")})
        # If(bcd == 0, self.char.eq(ord('0'))).Elseif().Elseif()...

# Let's convert this to Verilog!
from migen.fhdl.verilog import convert
bcd = Signal(4)      # input signal
m = bcd2ascii_case(bcd) # module instance
print(convert(m, ios={m.char, bcd}))

In [ ]:
# We can also use an array defined locally if we don't want the trouble of creating a module
bcd_ascii = Array(C(ord(c)) for c in "0123456789abcdef")

# Let's convert this to Verilog!
class top_bcd2ascii_array(Module):
    def __init__(self, bcd):
        self.char = bcd_ascii[bcd]
    
from migen.fhdl.verilog import convert
bcd = Signal(4)      # input signal
m = bcd2ascii_case(bcd) # module instance
vlog = convert(m, ios={m.char, bcd})
display(verilog2blocsvg(vlog))
print(vlog)

# LiteX
'System on Chip' generator
* builds over Migen
* provides standard building blocks, and ways to put them together (CSR, memories, streams, bus arbiter, PCIe, SATA, SD, UART, ...) in a SoC
* generates a Verilog model of your SoC
* generates Software headers (.c/.h/.csv) for your peripherals
* calls the appropriate FPGA toolchain
* can build your Software aswell
* has tools to help you debug your SoC (like probing CSR, scoping signals)
* helps documenting your peripherals

`python -m pip install --user git+https://github.com/enjoy-digital/litex.git`

![litex_netv](https://raw.githubusercontent.com/enjoy-digital/netv2/master/doc/architecture.png)

# ULX3S
Simple board based on Lattice's ECP5
* [mainpage](https://ulx3s.github.io/)
![ulx3s](https://ulx3s.github.io/images/ulx3s-v303-ax-top_png_project-main.jpg)

# Litex ULX3S 'hello world'
<strike>Install litex-boards for ULX3S support-

`python -m pip install --user git+https://github.com/litex-hub/litex-boards.git`</strike> not required for ulx3s, but if yours is not natively supported in LiteX, have a look here

Install `ujprog` (original work https://github.com/f32c/tools but needs patches)
```
git clone https://github.com/chmousset/f32c-tools.git
cd f32c-tools/ujprog
mkdir build ; cd build
cmake ..
make && make install
```

In [ ]:
# A bit of boilerplate...
from migen import * # the FHDL
from litex import * # the SoC builder
from litex.build.generic_platform import * # so we can use Pins(), etc to extend the platform
# the Platform represents the ulx3s 'capabilities', which FPGA brand/type is uses etc
from litex.boards.platforms.ulx3s import Platform

#import os
#os.makedirs('build')

def buildplat():
    # by default, LiteX uses Lattice's closed-source toolchain.
    # ULX3S has 4 FPGA size variants, set 'device' accordingly
    plat = Platform(toolchain="trellis", device="LFE5U-12F")

    # these IOs should probably be defined by default in litex, but since it's not we have to do it
    plat.add_extension([
        ("user_button", 0, Pins("D6"), IOStandard("LVCMOS33")),  # BTN_PWRn (inverted logic)
        ("user_button", 1, Pins("R1"), IOStandard("LVCMOS33")),  # FIRE1
        ("user_button", 2, Pins("T1"), IOStandard("LVCMOS33")),  # FIRE2
        ("user_button", 3, Pins("R18"), IOStandard("LVCMOS33")), # UP
        ("user_button", 4, Pins("V1"), IOStandard("LVCMOS33")),  # DOWN
        ("user_button", 5, Pins("U1"), IOStandard("LVCMOS33")),  # LEFT
        ("user_button", 6, Pins("H16"), IOStandard("LVCMOS33")), # RIGHT
    ])
    return plat

In [ ]:
# DEMO 1: LED display state of push buttons
class hello(Module):
    def __init__(self, plat):
        self.platform = plat
        self.ios = set()
        for i in range(7):
            led = plat.request("user_led", i)
            btn = plat.request("user_button", i)
            self.comb += led.eq(btn)
            self.ios = self.ios | {led, btn}

plat = buildplat()
top = hello(plat)
vlog = convert(top, ios=top.ios)
display(verilog2blocsvg(vlog))
print(vlog)

In [ ]:
 
plat = buildplat()
b = plat.build(hello(plat), run=True)
prog = plat.create_programmer().load_bitstream('build/top.bit')

In [ ]:
# DEMO 2: Binary counter. LED output MSB of the counter
class counter(Module):
    def __init__(self, plat):
        self.platform = plat
        counter = Signal(28) # we add 21 bits to slow down the LED frequencies
        self.sync += counter.eq(counter + 1) # counter increments each sys_clk rising edge
        self.ios = set()
        for i in range(8):
            led = plat.request("user_led", i)
            self.comb += led.eq(counter[-1-i]) # LED represent only the 8 MSb of the counter
            self.ios = self.ios | {led}
            # self.comb += plat.request("user_led", i).eq(counter[-1-i])

plat = buildplat()
top = counter(plat)
vlog = convert(top, top.ios)
display(verilog2blocsvg(vlog))
print(vlog)

plat = buildplat()
b = plat.build(counter(plat), run=True)

In [ ]:
# Let's see how it looks before packing & routing
ys = """
    read_verilog {path}/build/top.v
    proc
    opt
    show
    """.format(path=os.getcwd())
a = !yosys {str2tmpfile(ys, '.ys')}

In [ ]:
# Let's see how it looks inside the FPGA
ys = """
    read_verilog {path}/build/top.v
    synth_ecp5   -json top.json -top top
    show
    """.format(path=os.getcwd())
a = !yosys {str2tmpfile(ys, '.ys')}

In [ ]:
prog = plat.create_programmer().load_bitstream('build/top.bit')


# More ressources
 * Migen Manual : [https://m-labs.hk/migen/manual/](https://m-labs.hk/migen/manual/)
 * Migen Examples [https://github.com/m-labs/migen/tree/master/examples/basic](https://github.com/m-labs/migen/tree/master/examples/basic)

In [ ]:
import qrcode
qrcode.make("https://github.com/chmousset/fpga_workshop")